In [2]:
import tensorflow as tf
from scipy.spatial.transform import Rotation as R
from keras.layers import Input, Dense
from keras.models import Model
from keras import models
from keras import layers
from keras import optimizers
from keras import losses

import pandas as pd
import matplotlib 
import csv
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Read the CSV file from a subdirectory within your Google Drive
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/output_file.csv')


Mounted at /content/drive


In [12]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

import csv
import numpy as np

# Read data from the CSV file
with open('/content/drive/MyDrive/Colab Notebooks/output_file.csv', 'r') as f:
    reader = csv.reader(f)
    data = np.array([list(map(float, row[:12])) for row in reader])

# Use slicing to separate the first 9 values from the last 3
first_9 = data[:, :9]  # Get all rows, first 9 columns
place = data[:, 9:]
data=first_9   # Get all rows, last 3 columns
# Convert rotation matrices to Euler angles
euler_angles = []
for i in range(data.shape[0]):
    R = data[i].reshape(3, 3)
    sy = np.sqrt(R[0,0]**2 + R[1,0]**2)
    if sy > 1e-6:
        x = np.arctan2(R[2,1], R[2,2])
        y = np.arctan2(-R[2,0], sy)
        z = np.arctan2(R[1,0], R[0,0])
    else:
        x = np.arctan2(-R[1,2], R[1,1])
        y = np.arctan2(-R[2,0], sy)
        z = 0
    euler_angles.append([x, y, z])


# Normalize coordinates using min-max scaling
min_coord = np.min(place, axis=0)
max_coord = np.max(place, axis=0)
normalized_coordinates = (place - min_coord) / (max_coord - min_coord)


euler_angles = np.degrees(euler_angles)

mean_angle = np.mean(euler_angles, axis=0)
std_angle = np.std(euler_angles, axis=0)
normalized_angles = (euler_angles - mean_angle) / std_angle
c = np.concatenate((place, normalized_angles), axis=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
def dh_matrix(a, alpha, d, theta):
   
  ct = np.cos(theta)
  st = np.sin(theta)
  ca = np.cos(alpha)
  sa = np.sin(alpha)

  T = np.array([[ct, -st*ca, st*sa, a*ct],[st, ct*ca, -ct*sa, a*st], [0, sa, ca, d], [0, 0, 0, 1]])

  return T

In [14]:
def fkine(dh_params):
    
    T = np.eye(4)
    for i in range(len(dh_params)):
        T_i = dh_matrix(dh_params[i][0], dh_params[i][1], dh_params[i][2], dh_params[i][3])
        T = np.dot(T, T_i)

    pos = T[:3, 3]
    R_mat = T[:3, :3]
    # Convert the rotation matrix to a rotation object
    R_obj = R.from_matrix(R_mat)

    # Get the Euler angles (in degrees)
    euler_angles = R_obj.as_euler('xyz', degrees=True)
    euler_array = np.array(euler_angles)
    arr = np.concatenate((pos,euler_angles))
    return arr

In [15]:
with open('/content/drive/MyDrive/Colab Notebooks/theta_file.csv', 'r') as f:
    reader = csv.reader(f)
    ya= np.array([list(map(float, row[:6])) for row in reader])
print(ya)

[[ 42.008   9.     67.    -49.     86.    -71.65 ]
 [ 37.483  29.     22.     71.     85.    129.1  ]
 [ 23.79   18.     57.    104.     77.    137.53 ]
 ...
 [ 24.743  36.     28.    144.     81.    -51.329]
 [ 48.588  18.     54.    106.     38.    -87.002]
 [ 40.595  15.     36.    -74.     99.    -87.446]]


In [16]:
def custom_loss(y_true,theta):
    a = [0, 431.8, 20.30, 0, 0, 0]
    alpha = [-np.pi/2, 0, np.pi/2, -np.pi/2, np.pi/2, 0]
    d = [671.83, 139.70, 0, 431, 0, 56.50]
    theta=[44.99,43.00,46,44,46,45.98]
    theta = np.array(theta) * np.pi / 180
    y_pred=fkine(list(zip(a, alpha, d, theta)))
    loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return loss


In [18]:
# Define the input shape
input_shape = (6,)

# Define the model architecture
inputs = Input(shape=input_shape)
x = Dense(32, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(6)(x)

# Define the model
model = Model(inputs=inputs, outputs=outputs)


# compile the model with the normal mean squared error loss function
model.compile(optimizer=optimizers.Adam(), loss=losses.mean_squared_error)



# Get the number of columns
num_cols = c.shape[1]

# Reshape the array into a new shape (N, M)
c = c.reshape((-1, num_cols))


# Train the model on the input data
model.fit(x=c, y=ya, epochs=10000, batch_size=32)


Epoch 1/1000
55/55 [==============================] - 2s 2ms/step - loss: 3661.5425
Epoch 2/1000
55/55 [==============================] - 0s 2ms/step - loss: 3059.5166
Epoch 3/1000
55/55 [==============================] - 0s 2ms/step - loss: 3050.9238
Epoch 4/1000
55/55 [==============================] - 0s 2ms/step - loss: 3037.4583
Epoch 5/1000
55/55 [==============================] - 0s 2ms/step - loss: 3041.8115
Epoch 6/1000
55/55 [==============================] - 0s 2ms/step - loss: 3036.4971
Epoch 7/1000
55/55 [==============================] - 0s 2ms/step - loss: 3048.1846
Epoch 8/1000
55/55 [==============================] - 0s 3ms/step - loss: 3037.0039
Epoch 9/1000
55/55 [==============================] - 0s 3ms/step - loss: 3034.5276
Epoch 10/1000
55/55 [==============================] - 0s 3ms/step - loss: 3029.3882
Epoch 11/1000
55/55 [==============================] - 0s 2ms/step - loss: 3035.8916
Epoch 12/1000
55/55 [==============================] - 0s 2ms/step - loss: